In [58]:
import re

import advent_of_code

REGISTER_RE = re.compile(r'^Register\s+(?P<name>[A-C]):\s+(?P<value>[0-9]+)$')
PROGRAM_RE = re.compile(r'^Program:\s+(?P<program>[0-7,]+)$')

state = {
    'instruction_pointer': 0,
    'outputs': []
}
for line in advent_of_code.load_data('day17.txt'):
    if not line:
        continue

    match = REGISTER_RE.fullmatch(line)
    if match:
        state[match['name']] = int(match['value'])
        continue

    match = PROGRAM_RE.fullmatch(line)
    if match:
        state['program'] = tuple(map(int, match['program'].split(',')))

def combo_operand(operand, state):
    return (0, 1, 2, 3, state['A'], state['B'], state['C'])[operand]

def adv(operand, state):
    try:
        state['A'] = state['A'] // 2 ** combo_operand(operand, state)
    except IndexError:
        return False

    state['instruction_pointer'] += 2
    return True

def bxl(operand, state):
    state['B'] = state['B'] ^ operand
    state['instruction_pointer'] += 2
    return True

def bst(operand, state):
    try:
        state['B'] = combo_operand(operand, state) % 8
    except IndexError:
        return False
    
    state['instruction_pointer'] += 2
    return True

def jnz(operand, state):
    if state['A'] == 0:
        state['instruction_pointer'] += 2
    else:
        state['instruction_pointer'] = operand
    return True

def bxc(_operand, state):
    state['B'] = state['B'] ^ state['C']
    state['instruction_pointer'] += 2
    return True

def out(operand, state):
    try:
        state['outputs'].append(combo_operand(operand, state) % 8)
    except IndexError:
        return False

    state['instruction_pointer'] += 2
    return True

def bdv(operand, state):
    try:
        state['B'] = state['A'] // 2 ** combo_operand(operand, state)
    except IndexError:
        return False

    state['instruction_pointer'] += 2
    return True

def cdv(operand, state):
    try:
        state['C'] = state['A'] // 2 ** combo_operand(operand, state)
    except IndexError:
        return False

    state['instruction_pointer'] += 2
    return True

def run_instruction(opcode, operand, state):
    return (adv, bxl, bst, jnz, bxc, out, bdv, cdv)[opcode](operand, state)

def run(state):
    while True:
        instruction_pointer = state['instruction_pointer']
        try:
            opcode = state['program'][instruction_pointer]
            operand = state['program'][instruction_pointer + 1]
        except IndexError:
            return True
        if not run_instruction(opcode, operand, state):
            return False

saved = {'A': state['A'], 'B': state['B'], 'C': state['C'], 'instruction_pointer': 0}
run(state)
print(','.join(map(str, state['outputs'])))

7,5,4,3,4,5,3,4,6


1. 2,4 bst A -> B = A % 8
2. 1,1 bxl 1 -> B = B ^ 1
3. 7,5 cdv B -> C = A / 2**B
4. 1,5 bxl 5 -> B = B ^ 5
5. 4,3 bxc   -> B = B ^ C
6. 5,5 out B
7. 0,3 adv 3 -> A = A / 8
8. 3,0 jnz 0

- The program is a loop. The last instruction jumps back to the first instruction unless register A contains 0.
- The only instruction which sets the value in register A is the seventh which divides by 8 (shifts three bits right)
- The only output instruction is the sixth
- The values in registers B and C going into each iteration don't matter as they are not used within the loop (B is
initialised from A in the first instruction and C is initialised from A and B in the third instruction)
- The program is 16 digits long so to output itself, it must loop 16 times. This means the initial value of A must be
longer than 21 bits but no longer than 24
- The value in register A for the last iteration of the loop is a most three bits. The value output is the same as the
program being run with that value as the input of A

Hence we can work backwards through the desired output to figure out what values we need in register A

In [59]:
def trial(state, saved, index):
    state.update(saved)
    state['A'] = index
    state['outputs'] = []
    if run(state):
        print(index, ','.join(map(str, state['outputs'])))

# Want an output of 0
for i in range(8):
    trial(state, saved, i)

0 4
1 4
2 6
3 7
4 0
5 1
6 2
7 3


In [60]:
# Only A=4 results in an output of 0

# Want an output of 3,0
for i in range(8):
    trial(state, saved, i + 4 * 8)

32 4,0
33 4,0
34 2,0
35 7,0
36 1,0
37 3,0
38 2,0
39 3,0


In [61]:
# Only A=37 and A=39 result in an output of 3,0

# Want an output of 3,3,0
for i in range(8):
    trial(state, saved, i + 37 * 8)
for i in range(8):
    trial(state, saved, i + 39 * 8)

296 0,3,0
297 4,3,0
298 3,3,0
299 5,3,0
300 1,3,0
301 3,3,0
302 0,3,0
303 7,3,0
312 0,3,0
313 4,3,0
314 1,3,0
315 1,3,0
316 1,3,0
317 2,3,0
318 0,3,0
319 7,3,0


In [62]:
# Only A=298 and A=301 result in an output of 3,3,0

# Want an output of 0,3,3,0
for i in range(8):
    trial(state, saved, i + 298 * 8)
for i in range(8):
    trial(state, saved, i + 301 * 8)

2384 4,3,3,0
2385 4,3,3,0
2386 4,3,3,0
2387 3,3,3,0
2388 2,3,3,0
2389 4,3,3,0
2390 0,3,3,0
2391 6,3,3,0
2408 0,3,3,0
2409 4,3,3,0
2410 3,3,3,0
2411 5,3,3,0
2412 3,3,3,0
2413 7,3,3,0
2414 0,3,3,0
2415 6,3,3,0


In [63]:
# Only A=2390, A=2408 and A=2414 result in an output of 0,3,3,0

# Want an output of 5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 2390 * 8)
for i in range(8):
    trial(state, saved, i + 2408 * 8)
for i in range(8):
    trial(state, saved, i + 2414 * 8)

19120 4,0,3,3,0
19121 4,0,3,3,0
19122 0,0,3,3,0
19123 3,0,3,3,0
19124 5,0,3,3,0
19125 2,0,3,3,0
19126 7,0,3,3,0
19127 1,0,3,3,0
19264 4,0,3,3,0
19265 4,0,3,3,0
19266 6,0,3,3,0
19267 7,0,3,3,0
19268 2,0,3,3,0
19269 5,0,3,3,0
19270 4,0,3,3,0
19271 6,0,3,3,0
19312 4,0,3,3,0
19313 4,0,3,3,0
19314 0,0,3,3,0
19315 3,0,3,3,0
19316 3,0,3,3,0
19317 6,0,3,3,0
19318 4,0,3,3,0
19319 6,0,3,3,0


In [64]:
# Only A=19124 and A=19269 result in an output of 5,0,3,3,0

# Want an output of 5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 19124 * 8)
for i in range(8):
    trial(state, saved, i + 19269 * 8)

152992 4,5,0,3,3,0
152993 4,5,0,3,3,0
152994 2,5,0,3,3,0
152995 7,5,0,3,3,0
152996 5,5,0,3,3,0
152997 3,5,0,3,3,0
152998 1,5,0,3,3,0
152999 5,5,0,3,3,0
154152 0,5,0,3,3,0
154153 4,5,0,3,3,0
154154 3,5,0,3,3,0
154155 5,5,0,3,3,0
154156 1,5,0,3,3,0
154157 3,5,0,3,3,0
154158 6,5,0,3,3,0
154159 3,5,0,3,3,0


In [65]:
# Only A=152996 and A=154155 result in an output of 5,5,0,3,3,0

# Want an output of 3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 152996 * 8)
for i in range(8):
    trial(state, saved, i + 154155 * 8)

1223968 4,5,5,0,3,3,0
1223969 4,5,5,0,3,3,0
1223970 2,5,5,0,3,3,0
1223971 7,5,5,0,3,3,0
1223972 1,5,5,0,3,3,0
1223973 3,5,5,0,3,3,0
1223974 0,5,5,0,3,3,0
1223975 7,5,5,0,3,3,0
1233240 0,5,5,0,3,3,0
1233241 4,5,5,0,3,3,0
1233242 5,5,5,0,3,3,0
1233243 1,5,5,0,3,3,0
1233244 2,5,5,0,3,3,0
1233245 4,5,5,0,3,3,0
1233246 0,5,5,0,3,3,0
1233247 6,5,5,0,3,3,0


In [66]:
# Only A=1223973 results in an output of 3,5,5,0,3,3,0

# Want an output of 4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 1223973 * 8)

9791784 0,3,5,5,0,3,3,0
9791785 4,3,5,5,0,3,3,0
9791786 3,3,5,5,0,3,3,0
9791787 5,3,5,5,0,3,3,0
9791788 1,3,5,5,0,3,3,0
9791789 3,3,5,5,0,3,3,0
9791790 0,3,5,5,0,3,3,0
9791791 7,3,5,5,0,3,3,0


In [67]:
# Only A=9791785 results in an output of 4,3,5,5,0,3,3,0

# Want an output of 5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 9791785 * 8)

78334280 0,4,3,5,5,0,3,3,0
78334281 4,4,3,5,5,0,3,3,0
78334282 7,4,3,5,5,0,3,3,0
78334283 5,4,3,5,5,0,3,3,0
78334284 2,4,3,5,5,0,3,3,0
78334285 5,4,3,5,5,0,3,3,0
78334286 0,4,3,5,5,0,3,3,0
78334287 6,4,3,5,5,0,3,3,0


In [68]:
# Only A=78334283 and A=78334285 result in an output of 5,4,3,5,5,0,3,3,0

# Want an output of 1,5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 78334283 * 8)
for i in range(8):
    trial(state, saved, i + 78334285 * 8)

626674264 0,5,4,3,5,5,0,3,3,0
626674265 4,5,4,3,5,5,0,3,3,0
626674266 5,5,4,3,5,5,0,3,3,0
626674267 1,5,4,3,5,5,0,3,3,0
626674268 2,5,4,3,5,5,0,3,3,0
626674269 4,5,4,3,5,5,0,3,3,0
626674270 6,5,4,3,5,5,0,3,3,0
626674271 2,5,4,3,5,5,0,3,3,0
626674280 0,5,4,3,5,5,0,3,3,0
626674281 4,5,4,3,5,5,0,3,3,0
626674282 3,5,4,3,5,5,0,3,3,0
626674283 5,5,4,3,5,5,0,3,3,0
626674284 3,5,4,3,5,5,0,3,3,0
626674285 7,5,4,3,5,5,0,3,3,0
626674286 6,5,4,3,5,5,0,3,3,0
626674287 2,5,4,3,5,5,0,3,3,0


In [69]:
# Only A=626674267 results in an output of 1,5,4,3,5,5,0,3,3,0

# Want an output of 5,1,5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 626674267 * 8)

5013394136 0,1,5,4,3,5,5,0,3,3,0
5013394137 4,1,5,4,3,5,5,0,3,3,0
5013394138 5,1,5,4,3,5,5,0,3,3,0
5013394139 1,1,5,4,3,5,5,0,3,3,0
5013394140 6,1,5,4,3,5,5,0,3,3,0
5013394141 4,1,5,4,3,5,5,0,3,3,0
5013394142 7,1,5,4,3,5,5,0,3,3,0
5013394143 0,1,5,4,3,5,5,0,3,3,0


In [70]:
# Only A=5013394138 results in an output of 5,1,5,4,3,5,5,0,3,3,0

# Want an output of 7,5,1,5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 5013394138 * 8)

40107153104 4,5,1,5,4,3,5,5,0,3,3,0
40107153105 4,5,1,5,4,3,5,5,0,3,3,0
40107153106 4,5,1,5,4,3,5,5,0,3,3,0
40107153107 3,5,1,5,4,3,5,5,0,3,3,0
40107153108 6,5,1,5,4,3,5,5,0,3,3,0
40107153109 4,5,1,5,4,3,5,5,0,3,3,0
40107153110 7,5,1,5,4,3,5,5,0,3,3,0
40107153111 0,5,1,5,4,3,5,5,0,3,3,0


In [71]:
# Only A=40107153110 results in an output of 7,5,1,5,4,3,5,5,0,3,3,0

# Want an output of 1,7,5,1,5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 40107153110 * 8)

320857224880 4,7,5,1,5,4,3,5,5,0,3,3,0
320857224881 4,7,5,1,5,4,3,5,5,0,3,3,0
320857224882 0,7,5,1,5,4,3,5,5,0,3,3,0
320857224883 3,7,5,1,5,4,3,5,5,0,3,3,0
320857224884 5,7,5,1,5,4,3,5,5,0,3,3,0
320857224885 2,7,5,1,5,4,3,5,5,0,3,3,0
320857224886 7,7,5,1,5,4,3,5,5,0,3,3,0
320857224887 1,7,5,1,5,4,3,5,5,0,3,3,0


In [72]:
# Only A=320857224887 results in an output of 1,7,5,1,5,4,3,5,5,0,3,3,0

# Want an output of 1,1,7,5,1,5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 320857224887 * 8)

2566857799096 0,1,7,5,1,5,4,3,5,5,0,3,3,0
2566857799097 4,1,7,5,1,5,4,3,5,5,0,3,3,0
2566857799098 1,1,7,5,1,5,4,3,5,5,0,3,3,0
2566857799099 1,1,7,5,1,5,4,3,5,5,0,3,3,0
2566857799100 5,1,7,5,1,5,4,3,5,5,0,3,3,0
2566857799101 2,1,7,5,1,5,4,3,5,5,0,3,3,0
2566857799102 1,1,7,5,1,5,4,3,5,5,0,3,3,0
2566857799103 5,1,7,5,1,5,4,3,5,5,0,3,3,0


In [73]:
# Only A=2566857799098, A=2566857799099 and A=2566857799102 results in an output of 1,1,7,5,1,5,4,3,5,5,0,3,3,0

# Want an output of 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 2566857799098 * 8)
for i in range(8):
    trial(state, saved, i + 2566857799099 * 8)
for i in range(8):
    trial(state, saved, i + 2566857799102 * 8)

20534862392784 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392785 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392786 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392787 3,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392788 6,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392789 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392790 1,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392791 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392792 0,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392793 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392794 5,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392795 1,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392796 6,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392797 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392798 1,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392799 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392816 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392817 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392818 0,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392819 3,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392820 7,1,1,7,5,1,5,4,3,5,5,0,3,3,0
20534862392821 6,1,1,7,5,1,5,4,3,5,5,0,3,3,0
2053486239

In [74]:
# There are eleven values of A which result in an output of 4,1,1,7,5,1,5,4,3,5,5,0,3,3,0

# Want an output of 2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
for i in range(8):
    trial(state, saved, i + 20534862392784 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392785 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392786 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392789 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392791 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392793 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392797 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392799 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392816 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392817 * 8)
for i in range(8):
    trial(state, saved, i + 20534862392823 * 8)

164278899142272 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142273 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142274 6,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142275 7,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142276 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142277 1,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142278 7,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142279 1,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142280 0,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142281 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142282 7,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142283 5,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142284 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142285 1,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142286 7,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142287 1,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142288 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142289 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142290 4,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142291 3,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0
164278899142292 4,4,1,1,7,5,1,5,4,3,5,5,

164278899142333 is the smallest value of A which results in an output of 2,4,1,1,7,5,1,5,4,3,5,5,0,3,3,0